# Isenberg School of Management Hub

__Concept__

The project presented in this notebook is an adaptation of the Isenberg School of Management Hub, originally designed by BIG architect as an extension to the old school building in Amherst, Massachusetts, USA. Its most prominent feature is the domino shaped façade, which results from a progressive tilting of the façade's beams. The program elaborated here is a personal interpretation of the original design with some creative liberty in what regards the parametric variations.

__Notebook Structure__

The notebook is organized by constructive elements. Each section explains how we modeled each part of the building, both the concept and the way the functions are implemented. The final section explores the parametric variations allowed by the program.

__Running the Notebook__ 

This Algorithmic Design (AD) program was written in the Julia programming language and using the Khepri AD tool. Please install the requited dependencies (used packaged listed below) in order to run the notebook locally on your PC.

| Exterior render                                       | Entrance view                                         |       
|-------------------------------------------------------|-------------------------------------------------------|
| <img src="./figures/render_sunset.png" width="450"> | <img src="./figures/render_entrance.png" width="450"> | 

| Render of the lobby                                | Arerial view                                       |
|---------------------------------------------------|----------------------------------------------------|
| <img src="./figures/render_loby.png" width="410"> | <img src="./figures/render_beams.png" width="490"> | 

| | | Sketches made during the development of this program |
|-------------|--------------|-----------------|
| <img src="./drawings/S1.jpg" width="300"> | <img src="./drawings/S2.jpg" width="300"> |  <img src="./drawings/d1.jpg" width="280"> | 

| | |
|------------|----------------|
| <img src="./drawings/d2.jpg" width="300"> | <img src="./drawings/S3.jpg" width="500"> |

| Construction sequence                                 | 
|-------------------------------------------------------|
| <img src="./figures/gen_sequence_rev.gif" width="900"> |
| Slabs ---- Straight beams ---- Tilted beams ---- Exterior walls ---- Interior pillars ---- Interior walls ---- Façade curtain walls ---- Tilted glass panels |

# Packages

__Install:__

__Use:__

In [1]:
using WebIO

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3107053204923560428\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8588206334165350998\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3107053204923560428\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
using Interact

In [3]:
using Khepri

In [4]:
using Logging

In [5]:
render_size(800, 400) # Khepri backends size

(800, 400)

__Outlining__

The following macro allows you to use outlining in this notebook. `avoid_tests`, when set to true, will allow you to run the run the entire notebook skipping intermediate tests. When set to false, you may run the tests spread through the document, which illustrate the program's functions.

In [6]:
avoid_tests = Parameter(true)

macro test(expr...)
  quote
    if !avoid_tests() 
        begin
            $(esc(expr...))
        end
    end
  end
end 

@test (macro with 1 method)

In [7]:
# avoid_tests(false)

__Remove logs:__ avoid log messages as cell outputs.

In [8]:
Logging.disable_logging(Logging.Info)

LogLevel(1)

# Family properties

This program used Khepri's BIM operations. This cell contains several BIM families whose defaults where modified to suit this project:

In [9]:
copper_wall_fam = wall_family_element(default_wall_family())
frame_width = 0.1
frame_fam = column_family_element(default_column_family(), profile=rectangular_profile(frame_width, frame_width))
pillar_fam = column_family_element(default_column_family(), profile=circular_profile(0.2))
door_fam = door_family_element(default_door_family())
ground_fam = slab_family_element(default_slab_family());

# Slabs

__Parameters explained__

`pts_circle`

* center = center of the circle
* r = radius of the circle
* alfa_init = beginning angle for the circular distribution of points
* alfa_end = ending angle for the circular distribution of points
* n = number of points created

`isenberg_slab`

* ri = slab interior radius
* re = slab exterior radius
* alfa_proj = circle angle at which the building's columns start to tilt. At this point the slab deviates from the circular path to accompany their movement
* thick = slab thickness
* is_first = boolean value: is it the first slab? The base slab of the building is the only one whose shape changes at the alfa_proj angle to accompany the tilted columns

`slabs`

* floor_h = floor height
* floors = number of floors of the building


| Deault slab                           | First slab                  |
|--------------------------------------------------------|------------------------------------------------------|
| <img src="./figures/slabs_up.png" width="350"> | <img src="./figures/slabs_1srt.png" width="430"> |

This function distributes points in a circular fashion:

In [10]:
pts_circle(center, r, alfa_init, alfa_end, n)=
    [center+vpol(r, alfa) for alfa in division(alfa_init, alfa_end, n)]

pts_circle (generic function with 1 method)

In [11]:
@test begin

    backend(notebook)
    new_backend()

    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    r = widget(0:0.1:10, label="Circle radius")
                    delete_all_shapes()
                    pts = pts_circle(u0(), r, alfa_init, alfa_end, 20)
                    spline(pts)
    end
    
end 

Expected result:

<img src="./plots/pts_circle_sliders.png" width="800">

This function creates one circular slab only. The slab has a C shape, with 2 lists of circular locations as contour:

In [12]:
isenberg_slab(center, ri, re, alfa_init, alfa_proj, alfa_end, n, is_first=false) =
    let pts_in = pts_circle(center, ri, alfa_init, alfa_end, n)
        pts_out = pts_circle(center, re, alfa_init, alfa_end, n)
        pts_out_proj = pts_circle(center, re, alfa_init, alfa_proj, n)
        p_tang = center+vcyl(re, alfa_proj, 0) # tangent point at the beginig of the projection
        amplitude = alfa_end-alfa_proj
        distance = amplitude*re/(pi/2) # rule of 3 for the distance
        v = vpol(distance, alfa_proj + amplitude) # normal the initial angle of the projection
        is_first ?
        line([pts_out_proj..., p_tang+v, pts_out[end], reverse(pts_in)..., pts_out_proj[1]]) :
                line([pts_out..., reverse(pts_in)..., pts_out[1]])
    end

isenberg_slab (generic function with 2 methods)

In [13]:
@test begin

    backend(notebook)
    new_backend()
    
    @manipulate for alfa_init = widget(0:pi/20:pi/2, label="Initial angle"),
                    alfa_proj = widget(pi/4:pi/20:3pi/2, label="Projection angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:10, label="Outer radius")
                    delete_all_shapes()
                    isenberg_slab(u0(), ri, re, alfa_init, alfa_proj, alfa_end, 30)
    end
    
end 

Expected result:
<img src="./plots/isenberg_slab_false.png" width="800">

In [14]:
@test begin

    backend(notebook)
    new_backend()
    
    @manipulate for alfa_init = widget(0:pi/20:pi/2, label="Initial angle"),
                    alfa_proj = widget(pi/4:pi/20:3pi/2, label="Projection angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:10, label="Outer radius")
                    delete_all_shapes()
                    isenberg_slab(u0(), ri, re, alfa_init, alfa_proj, alfa_end, 30, true)
    end
    
    
end 

Expected result:
<img src="./plots/isenberg_slab_true.png" width="800">

In [15]:
isenberg_slab(center, ri, re, alfa_init, alfa_proj, alfa_end, thick, n, is_first=false) =
    let level = cz(center),
        center = center-vz(level),
        pts_in = pts_circle(center, ri, alfa_init, alfa_end, n)
        pts_out = pts_circle(center, re, alfa_init, alfa_end, n)
        pts_out_proj = pts_circle(center, re, alfa_init, alfa_proj, n)
        p_tang = center+vcyl(re, alfa_proj, 0) # tangent point at the beginig of the projection
        amplitude = alfa_end-alfa_proj
        distance = amplitude*re/(pi/2) # rule of 3 for the distance
        v = vpol(distance, alfa_proj + amplitude) # normal the initial angle of the projection
        default_slab_family(slab_family_element(default_slab_family(), thickness=thick))
        is_first ?
            slab(closed_polygonal_path([pts_out_proj..., p_tang+v, pts_out[end], reverse(pts_in)...]), level) :
                slab(closed_polygonal_path([pts_out..., reverse(pts_in)...]), level)
    end

isenberg_slab (generic function with 3 methods)

This function distributes circular slabs along the building's height:

In [16]:
slabs(center, ri, re, alfa_init, alfa_proj, alfa_end, thick, floor_h, floors, n) =
    let h = floor_h
        isenberg_slab(center, ri, re, alfa_init, alfa_proj, alfa_end, thick, n, true)
        for i in 1:floors
            isenberg_slab(center+vz(h*i), ri, re, alfa_init, alfa_proj, alfa_end, thick, n, false)
        end
    end

slabs (generic function with 1 method)

In [17]:
@test begin

    backend(meshcat)
    new_backend()

    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:15, label="Outer radius"),
                    floor_h = widget(1:0.1:3, label="Floor Height"),
                    floors = widget(1:1:5, label="Number of floors")
                    delete_all_shapes()
                    slabs(u0(), ri, re, alfa_init, alfa_end-pi/2, alfa_end, 0.1, floor_h, floors, 30)
    end

end 

Expected result:
<img src="./plots/slabs.png" width="800">

# Beams

## Profiles

__Parameters explained__

`isenberg_beam`

* p1 & p2 points define the beam's axis
* dx & dy are the beam's profile width and height, respectively
* alfa = profile rotation around the beam's axis
* bool = true for small beams, which use a centered rectangular profile
* bool = false for tilted/large beams, which use a bottom-aligned rectangular profile

This function creates a special profile to use in column or beam families. The rectangular profile will always be placed above the column/beam's axis:

In [18]:
bottom_aligned_rectangular_profile(Width::Real=1, Height::Real=1; width::Real=Width, height::Real=Height) =
  rectangular_path(xy(-width/2, 0), width, height)

bottom_aligned_rectangular_profile (generic function with 3 methods)

This function creates a beam for the Isenberg project. Given two random location is space, it places a free_columns object between them, with a profile of dx width and dy height, and rotated alfa radians. The boolean value decides the family that will be used (centered profile or bottom aligned profile):

In [19]:
isenberg_beam(p1, p2, dx, dy, alfa, bool=true) =
    let fam = bool ?
               column_family_element(default_column_family(), profile=rectangular_profile(dx, dy)) :
                column_family_element(default_column_family(), profile=bottom_aligned_rectangular_profile(dx, dy))
        free_column(p1, p2, alfa, family=fam)
    end

isenberg_beam (generic function with 2 methods)

| Centered rectangular profile                           |    | Bottom-aligned rectangular profile                   |
|--------------------------------------------------------|    |------------------------------------------------------|
| <img src="./figures/centered_profile.png" width="300"> | <img width="100"> | <img src="./figures/bottom_profile.png" width="300"> |

This function creates a small Isenberg beam, using a rectangular profile centered in the beam's axis:

In [20]:
small_beam(p1, p2, alfa) = isenberg_beam(p1, p2, beam_width, s_beam_lenght, alfa)

small_beam (generic function with 1 method)

This function creates a large Isenberg beam, with variable profile height, using a bottom-aligned profile:

In [21]:
large_beam(p1, p2, dy) = isenberg_beam(p1, p2, beam_width, dy, pi, false)

large_beam (generic function with 1 method)

## Beam sets

__Parameters explained__

`outer_beams`

* center = center of the circle along which  the beams are placed
* h = beam height
* r = radius of the circle along which  the beams are placed
* alfa_init = beginning angle for the circular distribution
* alfa_end = ending angle for the circular distribution
* n = number of beams created from alfa_init to alfa_end

`outer_beam_locs`

* large beams have a varying profile heigh, which increased as the beams get progressevly tilted along the façade
* d_min = min beam profile height
* d_max = maximum profile height



| Outer/small beams                           |  
|--------------------------------------------------------| 
| <img src="./figures/outer_beams.png" width="800"> |

This function creates an array of N vertical small squared beams from alfa_init to alfa_end:

In [22]:
outer_beams(center, h, r, alfa_init, alfa_end, n) =
    let pts_base = pts_circle(center, r, alfa_init, alfa_end, n)
        pts_top = pts_circle(center+vz(h), r, alfa_init, alfa_end, n)
        alfas = division(alfa_init, alfa_end, n)
        for (p,q,alfa) in zip(pts_base, pts_top, alfas)
            small_beam(p, q, alfa)
        end
    end

outer_beams (generic function with 1 method)

In [23]:
@test begin
    
    backend(meshcat)
    new_backend()
    s_beam_lenght = .4
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:15, label="Outer radius"),
                    floor_h = widget(1:0.1:3, label="Floor Height"),
                    floors = widget(1:1:5, label="Number of floors"),
                    beams = widget(10:1:30, label="Number of beams")
        
                    delete_all_shapes()
                    slabs(u0(), ri, re, alfa_init, alfa_end-pi/2, alfa_end, 0.1, floor_h, floors, 30)
                    outer_beams(u0(), floor_h*floors, re, alfa_init, alfa_end, beams)
    end
end

Expected result:
<img src="./plots/small_beams.png" width="800">

__Point projection__

* p_tang is the base point of the last straight column created. From this point on, columns start to tilt and enlarge.
* the base points for the large beams - `pts_base_line`- are created by a simple for cycle: p_tang is successively moved along a tangent line crossing the circle at p_tang, until N locations have been created in the given distance.
* this distance is calculated by a rule of 3 for the amplitude. If the amplitude where pi/2 the distance would be the circle's radius. Gicen the current amplitude the rule of 3 calculates the distance.
* the top points for the large beams - `pts_top_proj`- are calculated by projecting the linear point below to the circle above. For that we use triangle equivalence rules.

The image below explains how we derived the function `proj_angle(dist, radius)`:

* alfa is calculated based on the circle's radius (R - fixed measure) and the current distance (D - the distance of the point to be projected). 
* For each of the distances (D's) we add a cylindrical vector (+vcyl) to the center point (C), with 3 parameters: vcyl(r, angle, h). 
* r is the circle raius. 
* The angle is the initial angle for this part of the circle (alfa_init) plus the calculated proj_angle
* h id the building height

| Tilted beams | How to project linear points to a circle |  
|---------------|-----------------------------------------| 
| <img src="./drawings/tilted_beams.jpg" width="250"> | <img src="./drawings/project_func.png" width="550"> |

| Straight to tilted beams | Isenberg section and plan |
|--------------------------|---------------------------|
| <img src="./figures/beams_snt.png" width="400"> | <img src="./figures/plan_section.png" width="400"> |

__Local function definitions__

`push` is the minimum distance we must set the locations apart from the beam's axis in order for the metal frames not to be swallowed by the beams

`move_list` moves points with a distance equivalent to the space between beams, minus the push

`mv` moves points (forward or backwards dependin on the op sign) with a distance equivalent to the push

In [24]:
outer_beam_locs(center, h, r, alfa_init, alfa_end, d_min, d_max, n)=
    let proj_angle(dist, radius) = asin(dist/radius)
        p_tang = center+vpol(r, alfa_init) # tangent point at the beginig of the projection
        amplitude = alfa_end-alfa_init
        distance = amplitude*r/(pi/2) # rule of 3 for the distance
        v = vpol(1, alfa_init + amplitude) # normal the initial angle of the projection
        push = d_min/2+frame_width/2 # for the metal frame to be visible (not inside the beams)
        move_list(lst) = map(p -> p+v*(distance/n-push), lst)
        mv(lst, op) = map(p -> p+v*op(push), lst)
        v_psi(p, q) = π/2 -π/10 - sph_psi(q - p)

        pts_base_line = [p_tang+v*d for d in division(0, distance, n)]
        pts_top_proj = [center+vcyl(r, alfa_init+proj_angle(d, distance), h) for d in division(0, distance, n)]

        current_backend() == autocad || current_backend() == rhino ?
            pts_glass = [p+vz(dy) for (p,dy) in zip(pts_base_line, division(d_min, d_max*.9, n))] :
            pts_glass = [p + vsph(dy, alfa_init, v_psi(p, q)) for (p, q, dy)
                 in zip(pts_base_line, pts_top_proj, division(1e-9, d_max - d_min, n))]

        pts_base_line_mv = mv(pts_base_line, +)
        pts_top_proj_mv = mv(pts_top_proj, +)
        pts_glass_mv = mv(pts_glass, +)
        pts_base_move = move_list(pts_base_line)
        pts_top_move = move_list(pts_top_proj)
        pts_glass_move = move_list(pts_glass)
        pts_top_proj_next = mv(pts_top_proj, -)

        #RETURN:
        [pts_base_line, pts_base_line_mv, pts_base_move,
        pts_top_proj, pts_top_proj_mv, pts_top_move,
        pts_glass_mv, pts_glass_move, pts_top_proj_next]
    end

outer_beam_locs (generic function with 1 method)

__Outer Beam Locations' Array__

This function calculates all the locations associated with the large beams. Several functions will feed from the array of arrays that this one returns. The output array contains:

* pts_base_line = bottom points for large beams
* pts_top_proj = top points for large beams


* pts_base_move = bottom points projected onto the next beam, will be used to create glass panels between beams
* pts_top_move = top points projected onto the next beam, will be used to create glass panels between beams


* pts_glass_mv = intermediate points (bottom point lifted in z) for glass panels
* pts_glass_move = intermediate points for galss panels projected onto the next beam


* pts_base_line_mv = bottom points slightly desviated forward, will be used to create glass panels between beams
* pts_top_proj_mv = top points slightly desviated forward, will be used to create glass panels between beams
* pts_top_proj_next = top points slightly desviated backward, will be used to close the glass panels between beams (need a last point repetition to create a closed panel

In [25]:
@test begin

    backend(notebook)
    new_backend()
    
    @manipulate for alfa_init = widget(0:pi/20:pi, label="Initial projection angle"),
                    alfa_end = widget(pi/2:pi/20:3pi/2, label="Final angle")
                    delete_all_shapes()
                    r=3
                    locs=outer_beam_locs(u0(), 2, r, alfa_init, alfa_end, 1, 2, 20)
        
                    line(pts_circle(u0(), r, alfa_init-pi, alfa_end, 20)) # base slab
                    line(locs[1]) # pts_base_line
                    line(locs[4]) # pts_top_proj
                    line.(locs[1], locs[4]) #beam lines
    end
  
end

Expected result:
<img src="./plots/proj_points_beams.png" width="800">

In [26]:
@test begin

    backend(notebook)
    new_backend()
    
    @manipulate for alfa_init = widget(0:pi/20:pi, label="Initial projection angle"),
                    alfa_end = widget(pi/2:pi/20:3pi/2, label="Final angle")
                    delete_all_shapes()
                    r=3
                    locs=outer_beam_locs(u0(), 2, r, alfa_init, alfa_end, 1, 2, 20)
        
                    line(pts_circle(u0(), r, alfa_init-pi, alfa_end, 20)) # base slab
                    line(locs[1]) # pts_base_line
                    line(locs[2]) # pts_base_line_mv
                    line(locs[3]) # pts_base_move
                    line(locs[4]) # pts_top_proj
                    line(locs[5]) # pts_top_proj_mv
                    line(locs[6]) # pts_top_move
                    line(locs[6]) # pts_glass_mv
                    line(locs[6]) # pts_glass_move
                    line(locs[6]) # pts_top_proj_nex
    end
  
end

Expected result:
<img src="./plots/proj_points_mvs.png" width="800">

This function creates beams with varying widths along two lists of points, which are provided to the function in one joint array:

In [27]:
large_outer_beams_gen(list, d_min, d_max, n)=
    let pts_base_line = list[1][2:end]
        pts_top_proj = list[4][2:end]
        dys = division(d_min, d_max, n)
        for (p, q, dy) in zip(pts_base_line, pts_top_proj, dys)
            large_beam(p, q, dy)
        end
    end

large_outer_beams_gen (generic function with 1 method)

This function provides `large_outer_beams_gen` with the two lists of points required to make the tilted beams of Isenberg:

In [28]:
large_outer_beams(center, h, r, alfa_init, alfa_end, d_min, d_max, n)=
    let lst = outer_beam_locs(center, h, r, alfa_init, alfa_end, d_min, d_max, n)
        large_outer_beams_gen(lst, d_min, d_max, n)
    end

large_outer_beams (generic function with 1 method)

In [29]:
@test begin

    backend(meshcat)
    new_backend()
    beam_width = .1
    @manipulate for alfa_init = widget(0:pi/20:pi, label="Initial projection angle"),
                    alfa_end = widget(pi/2:pi/20:3pi/2, label="Final angle")
                    delete_all_shapes()        
                    large_outer_beams(u0(), 3, 5, alfa_init, alfa_end, .2, 1, 10)
    end
  
end

Expected result:
<img src="./plots/beams.png" width="800">

In [30]:
@test begin
   
    backend(meshcat)
    new_backend()
    s_beam_lenght = .4
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:15, label="Outer radius"),
                    floor_h = widget(1:0.1:3, label="Floor Height"),
                    floors = widget(1:1:5, label="Number of floors"),
                    l_beams = widget(10:1:15, label="Number of large beams"),
                    s_beams = widget(10:1:60, label="Number of small beams")
        
                    delete_all_shapes()
                    slabs(u0(), ri, re, alfa_init, alfa_end-pi/2, alfa_end, 0.1, floor_h, floors, 30)
                    outer_beams(u0(), floor_h*floors, re, alfa_init, alfa_end-pi/2, s_beams)
                    large_outer_beams(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 1, l_beams)
    end
end

Expected result:
<img src="./plots/large_beams.png" width="800">

# Glass

## Round curtain wall

This function creates a round curtain wall from `alfa_init` to `alfa_end` angles, with `r` radius, covering the given number of `floors` starting from the `center`'s height:

In [31]:
straight_glass(center, r, alfa_init, alfa_end, floor_h, floors, n) =
    let pts_base = pts_circle(center-vz(center.z), r, alfa_init, alfa_end, n)
        amp = alfa_end-alfa_init
        fam = curtain_wall_family_element(default_curtain_wall_family(), max_panel_dx=amp*r/n, max_panel_dy=floor_h)
        curtain_wall(polygonal_path(pts_base), bottom_level=level(center.z), 
            top_level=level(center.z+floor_h*floors), family=fam)
    end

straight_glass (generic function with 1 method)

In [32]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for alfa_init = widget(0:pi/20:pi, label="Initial projection angle"),
                    alfa_end = widget(pi/2:pi/20:3pi/2, label="Final angle")
                    delete_all_shapes()        
                    straight_glass(u0(), 4, alfa_init, alfa_end, 1, 3, 5) 
    end
  
end

Expected result:
<img src="./plots/curtain_wall.png" width="800">

## Triangular curtain wall

Given a list of points (closed polygon vertices), this function creates a polygonal glass panel surrounded by a thin metal framing all around:

In [33]:
isenberg_panel(pts)=
    begin
        panel(pts, family=default_panel_family())
        for (p,q) in zip(pts, [pts[2:end]...,pts[1]])
            free_column(p,q, family=frame_fam)
        end
    end

isenberg_panel (generic function with 1 method)

In [34]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for radius = widget(1:1:5, label="Radius"),
                    sides = widget(1:1:10, label="Sides"),
                    angle = widget(1:1:20, label="Angle")
                    delete_all_shapes()        
                    isenberg_panel(regular_polygon_vertices(sides, x(-10), radius, angle))
                    isenberg_panel(regular_polygon_vertices(sides+2, x(0), radius, angle))
                    isenberg_panel(regular_polygon_vertices(sides+5, x(10), radius, angle))
                    end
  
end

Expected result:
<img src="./plots/glass_panels_polygons.png" width="800">

Given Isenberg's main parameters, this function calculates the corners of the triangular glass entrance, and places a curtain wall between them:

In [35]:
triangle_glass(center, h, r, alfa_init, alfa_end, d_min, d_max, n) =
    let p1 = center+vpol(r, alfa_end)
          p2 = p1+vz(h)
          p3 = outer_beam_locs(center, h, r, alfa_init, alfa_end, d_min, d_max, n)[1][end]
          v1 = (p3-p2)
          v2 = (p3-p1)
          v3 = (p1-p2)
          ps_top(m) = [p2 + v1*d for d in division(0, 1, m, false)]
          ps_bottom(m) = [p1 + v2*d for d in division(0, 1, m, false)]
          ps_side(m) = [p2 + v3*d for d in division(0, 1, m, false)]
       isenberg_panel([p1, p2, p3])
       for (p, q) in zip(ps_top(n), ps_bottom(n))
          free_column(p, q, family=frame_fam)
       end
       for (p, q) in zip(ps_top(6), ps_side(6))
           distance(p, q)<1.0e-15 ? nothing : free_column(p, q, family=frame_fam)
       end
      end

triangle_glass (generic function with 1 method)

In [36]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for h = widget(1:1:10, label="Building height")
                    delete_all_shapes()        
                    triangle_glass(u0(), h, 10, 0, pi/4, 1, 1, 10) 
    end
  
end

Expected result:
<img src="./plots/triangular_glass.png" width="400">

In [37]:
@test begin
   
    backend(meshcat)
    new_backend()
    s_beam_lenght = .4
    beam_width = .1
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:15, label="Outer radius"),
                    floor_h = widget(1:0.1:3, label="Floor Height"),
                    floors = widget(1:1:5, label="Number of floors"),
                    l_beams = widget(10:1:15, label="Number of large beams"),
                    s_beams = widget(10:1:60, label="Number of small beams")
        
                    delete_all_shapes()
                    slabs(u0(), ri, re, alfa_init, alfa_end-pi/2, alfa_end, 0.1, floor_h, floors, 30)
                    outer_beams(u0(), floor_h*floors, re, alfa_init, alfa_end-pi/2, s_beams)
                    large_outer_beams(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 1, l_beams)
                    straight_glass(u0(), ri, alfa_init, alfa_end, floor_h, floors, 30)
                    straight_glass(u0(), re, alfa_init, alfa_end, floor_h, floors, 30)
                    triangle_glass(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 1, 30)
    end
end

Expected result:
<img src="./plots/vert_glass2.png" width="800">

## Tilted façade panels

| Move points to create glass panels between beams |  
|--------------------------------------------------------| 
| <img src="./drawings/outer_beam_locs.jpg" width="300"> |

This function creates `isenberg_panel`s along the entire façade tilting, in between the large beams. It function uses the lists of points produced by the `outer_beam_locs` function:

In [38]:
large_beam_glass(center, h, r, alfa_init, alfa_end, d_min, d_max, n) =
    let beam_locs = outer_beam_locs(center, h, r, alfa_init, alfa_end, d_min, d_max, n)
    pts_base_line_mv = beam_locs[2][1:end-1] # e vertice
    pts_top_proj_mv = beam_locs[5][1:end-1] # a vertice
    pts_base_move = beam_locs[3][1:end-1] # f vertice
    pts_top_move = beam_locs[6][1:end-1] # b vertice
    pts_glass_mv = beam_locs[7][1:end-1] # d vertice
    pts_glass_move = beam_locs[8][1:end-1] # c vertice
    pts_top_proj_next = beam_locs[9][2:end] # next a vertice to make the triangular panel on top
        for (a,b,c,d,e,f,a2) in zip(pts_top_proj_mv, pts_top_move,
                                    pts_glass_move, pts_glass_mv,
                                    pts_base_line_mv, pts_base_move,
                                    pts_top_proj_next)
            isenberg_panel([a, b, c, d])
            isenberg_panel([c, d, e, f])
            isenberg_panel([a, b, a2])
        end
    end

large_beam_glass (generic function with 1 method)

In [39]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for h = widget(1:1:10, label="Building height"),
                    r = widget(5:1:10, label="Radius")
                    delete_all_shapes()        
                    large_beam_glass(u0(), h, r, 0, pi/2, 0.1, 3, 10) 
    end
  
end

Expected result:
<img src="./plots/glass_panels.png" width="800">

In [40]:
@test begin
   
    backend(meshcat)
    new_backend()
    s_beam_lenght = .4
    beam_width = .2
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    ri = widget(0:0.1:5, label="Inner radius"),
                    re = widget(7:0.1:15, label="Outer radius"),
                    floor_h = widget(1:0.1:3, label="Floor Height"),
                    floors = widget(1:1:5, label="Number of floors"),
                    l_beams = widget(10:1:15, label="Number of large beams"),
                    s_beams = widget(10:1:60, label="Number of small beams")
        
                    delete_all_shapes()
                    slabs(u0(), ri, re, alfa_init, alfa_end-pi/2, alfa_end, 0.1, floor_h, floors, 30)
                    outer_beams(u0(), floor_h*floors, re, alfa_init, alfa_end-pi/2, s_beams)
                    large_outer_beams(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 2, l_beams)
                    straight_glass(u0(), ri, alfa_init, alfa_end, floor_h, floors, 30)
                    straight_glass(u0(), re, alfa_init, alfa_end, floor_h, floors, 30)
                    triangle_glass(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 2, 30)
                    large_beam_glass(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 2, l_beams)
    end
end

Expected result:
<img src="./plots/tilted_beams.png" width="800">

# Inner pillars

| Pillar distribution inside the building: plan view |  
|--------------------------------------------------------| 
| <img src="./drawings/pillar_params.jpg" width="400"> |

This function creates a circular array of `n` cylindrical pillars:

In [41]:
pillar_line(center, z, h, r, alfa_init, alfa_end, n)=
    for alfa in division(alfa_init, alfa_end, n, false)[2:end]
        current_backend() == autocad || current_backend() == rhino ?
            cylinder(center+vcyl(r, alfa, z), 0.2, center+vcyl(r, alfa, z+h)) :
            column(xy(center.x,center.y)+vpol(r, alfa), 0, level(z), level(z+h), pillar_fam)
    end

pillar_line (generic function with 1 method)

In [42]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    h = widget(0:1:3, label="Pilar height"),
                    r = widget(1:0.1:5, label="Array Radius"),
                    z = widget(0:1:10, label="Level height"),
                    n = widget(0:1:20, label="Number of pillars")
                    delete_all_shapes()        
                    pillar_line(u0(), z, h, r, alfa_init, alfa_end, n) 
    end
end

Expected result:
<img src="./plots/pilar_line.png" width="800">

This function creates `m` circular arrays of `n` cylindrical pillars:

In [43]:
pillar_lines(center, z, h, inner_r, outer_r, alfa_init, alfa_end, n, m)=
    for r in division(inner_r, outer_r, m-1)
        pillar_line(center, z, h, r, alfa_init, alfa_end, n)
    end

pillar_lines (generic function with 1 method)

In [44]:
@test begin
    backend(meshcat)
    new_backend()
    pillar_lines(u0(), 0, 3, 5, 10, 0, pi, 10, 3) 
end

Expected result:
<img src="./plots/pilar_lines.png" width="800">

This function distributes `m` circular arrays of `n` cylindrical pillars by the number of `floor`'s provided:

In [45]:
pillars_z(center, inner_r, outer_r, alfa_init, alfa_end, n, m, floor_h, floors)=
    for z in division(0, floor_h*floors, floors, false)
        pillar_lines(center, z, floor_h, inner_r, outer_r, alfa_init, alfa_end, n, m)
    end

pillars_z (generic function with 1 method)

In [46]:
@test begin
   
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 10
    floor_h = 3
    floors = 3
    
    delete_all_shapes()
    slabs(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, floor_h, floors, 30)
    pillars_z(u0(), ri, re, alfa_init, alfa_end, 10, 3, floor_h, floors)

end

Expected result:
<img src="./plots/pilars_z.png" width="600">

In [47]:
@test begin
   
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    floors = 3
    l_beams = 12
    s_beams = 35
       
    delete_all_shapes()
    slabs(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, floor_h, floors, 30)
    outer_beams(u0(), floor_h*floors, re, alfa_init, alfa_end-pi/2, s_beams)
    large_outer_beams(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 3, l_beams)
    straight_glass(u0(), ri, alfa_init, alfa_end, floor_h, floors, 30)
    straight_glass(u0(), re, alfa_init, alfa_end, floor_h, floors, 30)
    triangle_glass(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 2, 30)
    large_beam_glass(u0(), floor_h*floors, re, alfa_end-pi/2, alfa_end, .2, 2, l_beams)
    pillars_z(u0(), ri+1, re-1, alfa_init, alfa_end, 15, 5, floor_h, floors)

end

Expected result:
<img src="./plots/pilars_in_building.png" width="800">

# Walls

| Room distribution scheme: plan for inner walls |  
|-------------------------------------------------------| 
| <img src="./drawings/in_wall_params.jpg" width="600"> |

## Outer walls

This function creates a wall between points `p1` and `p2` with height `h` and thickness `thick`. The resulting wall belongs to the `copper_wall_fam` family, which should be attributed a copper-brick-like material:

In [48]:
isenberg_wall(p1, p2, h, thick) =
  let vp = p2-p1,
      level_z = p1.z,
      (p1, p2) = (p1, p2).-vz(level_z)
    wall([p1, p2], bottom_level=level(p1.z), top_level=level(h), family=copper_wall_fam)
  end

isenberg_wall (generic function with 1 method)

## Transversal inner walls

__Parameters explained__

`wall_trans_set`

* center = center of the circle along which the beams are placed
* z = floor height
* alfa = angle at witch the transversal wall line is placed
* inner_r = building's inner radius
* outer_r = building's outer radius
* corridor_w = corridor width
* h = floor height
* p_offset = distance between the outer wall and the first line of pillars
* m_pillars = number of pillars across the floor (transversal to the façade)
* broken = boolean value for full length wall (if false) or broken by corridors (if true)

`wall_trans_floor`

* alfa_init = beginning angle for the circular distribution
* alfa_end = ending angle for the circular distribution
* n_pillars = number of pillars across the floor (parallel to the façade)

This function creates one set of walls transversal to the façade. Depending on the boolean value given to `broken` the function generates either (1) a big wall that crosses the floor from `p_out` to `p_wall_in` or (2) a broken wall across the floor, which corresponds to 3 aligned walls separated by 2 corridors. The former is used in the ground floor atriums only. The later scheme composes all upper floors.

In [49]:
wall_trans_set(center, z, alfa, inner_r, outer_r, corridor_w, h, p_offset, m_pillars, broken)=
    let p_in = center+vpol(inner_r, alfa)
        p_out = center+vpol(outer_r, alfa)
        v_in = vpol(1, alfa+pi)
        v_out = vpol(1, alfa)
        pillar_rs = division(inner_r+p_offset, outer_r-p_offset, m_pillars-1)
        room_size = Int(floor(m_pillars/3))
        p_wall_in = center +vpol(pillar_rs[1+room_size], alfa)
        p_wall_out = center +vpol(pillar_rs[end-room_size], alfa)
        p_wall_mid_in = p_wall_in+v_in*corridor_w
        p_wall_mid_out = p_wall_out+v_out*corridor_w

        with(default_level, level(z), default_level_to_level_height, h) do
            broken ?
                begin
                    wall([p_in, p_wall_mid_in])
                    wall([p_wall_in, p_wall_out])
                    wall([p_wall_mid_out, p_out])
                end :
                wall([p_out, p_wall_in])
        end
    end

wall_trans_set (generic function with 1 method)

In [50]:
@test begin
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    floors = 3
    l_beams = 12
    s_beams = 35
    n_pillars = 20
    m_pillars = 4
    
    delete_all_shapes()
    slabs(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, floor_h, 1, 30)
    wall_trans_set(u0(), 0, 0, ri, re, 2, floor_h, 1, m_pillars, false)
    wall_trans_set(u0(), 0, pi/4, ri, re, 2, floor_h, 1, m_pillars, true)
end

Expected result:
<img src="./plots/wall_trans.png" width="800">

This function creates `n_pillars` sets of walls transversal to the façade:

In [51]:
wall_trans_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, p_offset, n_pillars, m_pillars; broken=true)=
    for alfa in division(alfa_init, alfa_end, n_pillars, false)[2:end]
        wall_trans_set(center, z, alfa, inner_r, outer_r, corridor_w, h, p_offset, m_pillars, broken)
    end

wall_trans_floor (generic function with 1 method)

In [52]:
@test begin  
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    floors = 1
    l_beams = 12
    s_beams = 35
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
       
    delete_all_shapes()

    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    pillars_z(u0(), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors) 
    wall_trans_floor(u0(), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars, broken=false)

    isenberg_slab(x(35), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    pillars_z(x(35), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors) 
    wall_trans_floor(x(35), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars)
end

Expected result:
<img src="./plots/wall_plans.png" width="800">

In [53]:
@test begin  
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    floors = 3
    l_beams = 12
    s_beams = 35
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
       
    delete_all_shapes()
    slabs(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, floor_h, floors, 30)
    straight_glass(u0(), ri, alfa_init, alfa_end, floor_h, floors, 30)
    straight_glass(u0(), re, alfa_init, alfa_end, floor_h, floors, 30)
    pillars_z(u0(), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors)
    
    wall_trans_floor(u0(), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars, broken=false)
    wall_trans_floor(u0(), floor_h, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars)
    wall_trans_floor(u0(), floor_h*2, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars)
end

Expected result:
<img src="./plots/wall_trans_set.png" width="800">

## Curved inner walls

__Parameters explained__

`curve_wall`

* center = center of the arc path along which the wall is created
* z = level height at which the wall is created
* alfa_init = beginning angle for the arc path 
* alfa_end = ending angle for the arc path
* r = radius if the arc path
* h = floor height, corresponds to wall height as well

`doors`

* w = doors receive the host wall as parameter
* pts = a door of the `door_fam` family is placed in each of the points of this list 

`curve_wall_broken`

* n_corridors = number of rooms in the floor. There will be 2 transversal corridors, respectivle at 1/3 and 2/3 of this number. The corridors will break the curve wall to create passages, eliminating a room that might have been there instead
* has_doors = boolean value defining if this wall has doors or not

This function creates an arc-shaped wall:

In [54]:
curve_wall(center, z, alfa_init, alfa_end, r, h)=
    begin
        wall(arc_path(center, r, alfa_init, alfa_end-alfa_init), bottom_level=level(z), top_level=level(z+h))
    #     stroke(arc_path(center, r, alfa_init, alfa_end-alfa_init)) #debug
    end

curve_wall (generic function with 1 method)

In [55]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for alfa_init = widget(0:pi/20:3pi/2, label="Initial angle"),
                    alfa_end = widget(pi/2:pi/20:2pi, label="Final angle"),
                    h = widget(0:1:3, label="Pilar height"),
                    r = widget(1:0.1:5, label="Array Radius")
                    delete_all_shapes()        
                    curve_wall(u0(), 0, alfa_init, alfa_end, r, h)
    end
end

Expected result:
<img src="./plots/wall_curve.png" width="800">

`doors` function places doors in each location of the list within the given wall's axis.

`and_dist` is an auxiliary function to calculate the angular distance of an arc.

`no_doors` should be a global variable. If true no doors are placed (that is to say, no boolean operation is conducted). If false, all doors will be created in the building. This variable prevents errors in backends where boolean operations are yet under-developed, and it helps speed up the generation process in all others when the goal is not to generate the complete building model. Boolean operations are computational expensive and there are a lot of doors in the building.


In [56]:
doors(w, pts) = no_doors ? w : foreach(pt->add_door(w, pt, door_fam), pts)
ang_dist(r, ang) = ang*r

no_doors = true

true

Unlike the `curve_wall` function, which generates a continuous wall across the entire floor, this one creates a broken wall, or rather a sequence of walls separated by corridors. The __full length wall__ is used for the outer walls, and the __broken walls__ are used in the middle ones to open up passages between corridors.

In [57]:
curve_wall_broken(center, z, alfa_init, alfa_end, r, h, n_corridors, has_doors=false)=
    let amplitude = (alfa_end-alfa_init)/n_corridors
        alfas = division(alfa_init, alfa_end, n_corridors, false)
        mark1 = floor(Int, n_corridors*(1/3))
        mark2 = ceil(Int, n_corridors*(2/3))
        door_w = 1

        function curve_wall_with_doors(angs, n_doors)
            local_amp = angs[end]-angs[1]
            door_locs = [x(a*r+local_amp*r/n_doors-door_w*1.2)
                            for a in division(0, local_amp, n_doors, false)]
            has_doors ?
                doors(curve_wall(center, z, angs[1], angs[end], r, h), door_locs) :
                curve_wall(center, z, angs[1], angs[end], r, h)
        end

        g1 = alfas[2:mark1]
        g2 = alfas[mark1+1:mark2]
        g3 = alfas[mark2+1:end]
        curve_wall_with_doors(g1, length(g1)-1)
        curve_wall_with_doors(g2,length(g2)-1)
        curve_wall_with_doors(g3, length(g3)-1)
    end

curve_wall_broken (generic function with 2 methods)

In [58]:
@test begin  
    backend(autocad)
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
           
    delete_all_shapes()
    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    curve_wall_broken(u0(), 0, alfa_init, alfa_end, ri+3, floor_h, n_pillars)
    curve_wall_broken(u0(), 0, alfa_init, alfa_end, re-3, floor_h, n_pillars)
end

Expected result:
<img src="./plots/wall_curve_broken.png" width="400">

__Paremeters explained__

`curve_walls`

* inner_r = radius for the arc path of the inner most wall
* outer_r = radius for the arc path of the outer most wall
* corridor_w = all across corridors' width - this defines the distance between `p_mid_in` and `p_wall_in`
* n_pillars = number of pillars across the floor in the direction of the arc path
* m_pillars = number of pillars across the floor transversely (will define/influence the number of rooms)
* wall_thick = wall thickness

In [59]:
curve_walls(center, z, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, n_pillars, m_pillars, wall_thick)=
    let pillar_rs = division(inner_r, outer_r, m_pillars-1)
        room_size = Int(floor(m_pillars/3))
        r_in = pillar_rs[1+room_size]
        r_out = pillar_rs[end-room_size]
        r_in_corridor = r_in-corridor_w
        r_out_corridor = r_out+corridor_w

        door_locs(r, angs) = [x(ang+wall_thick/2) for ang in angs]
        angs_in = division(0, ang_dist(r_in_corridor, alfa_end-alfa_init), n_pillars, false)
        angs_out = division(0, ang_dist(r_out_corridor, alfa_end-alfa_init), n_pillars, false)

        doors(curve_wall(center, z, alfa_init, alfa_end, r_in_corridor, h), door_locs(r_in_corridor, angs_in))
        curve_wall_broken(center, z, alfa_init, alfa_end, r_in, h, n_pillars, false)
        curve_wall_broken(center, z, alfa_init, alfa_end, r_out, h, n_pillars, true)
        doors(curve_wall(center, z, alfa_init, alfa_end, r_out_corridor, h), door_locs(r_out_corridor, angs_out))
    end

curve_walls (generic function with 1 method)

In [60]:
@test begin  
    backend(autocad)
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
       
    delete_all_shapes()
    
    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    pillars_z(u0(), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors) 
    wall_trans_floor(u0(), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars)
    curve_walls(u0(), 0, alfa_init, alfa_end, ri+p_offset, re-p_offset, corridor_w, floor_h, n_pillars, m_pillars, 0.1)
end

Expected result:
<img src="./plots/wall_floor_plan_test.png" width="500">

This function joins the __curve walls__ and the __transversal walls__, creating a complete floor plan:

In [61]:
rooms_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, p_offset, n_pillars, m_pillars, wall_thick)=
    begin
        wall_trans_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, p_offset, n_pillars, m_pillars)
        curve_walls(center, z, alfa_init, alfa_end, inner_r+p_offset, outer_r-pillar_offset, corridor_w, h, n_pillars, m_pillars, wall_thick)
    end

rooms_floor (generic function with 1 method)

In [62]:
@test begin  
    backend(autocad)
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
     
    no_doors = false
    
    delete_all_shapes()
    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    pillars_z(u0(), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors) 
    rooms_floor(u0(), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars, 0.1)
end

Expected result: 

| Should be the same as the previous test but with doors | Parallel projection |
|--------------------------------------------------------|---------------------|
|<img src="./plots/wall_floor_plan_doors.png" width="400">|<img src="./plots/wall_floor_plan_doors2.png" width="600"> |

# Lights

`lights_rooms_floor` spreads point lights throughout the corridors and rooms of the building, in an arc-shaped grid that accompanies the shape of the plan. 

`lights_on` is another global variable, created for performance sake. When set to true, all building lights are created which deeply slows both the generation and the navigation process. My advice is to generate the entire building without lights and without doors, navigate through it like so and pick a view. Save the view set, and then generate the complete thing for render purposes only.

In [63]:
light(p) = pointlight(p, #=color range=12,=# intensity=.9)

lights_rooms_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, h, p_offset, n_pillars, m_pillars)=
    let amp_tot = alfa_end-alfa_init
        amp_room = amp_tot/n_pillars
        map_division((m,n) -> light(center+vcyl(m,n,z+.8h)),
                        inner_r+p_offset, outer_r-p_offset, m_pillars, # pillar_rs
                        alfa_init+amp_room/2, alfa_end+amp_room/2, n_pillars, false) # alfas
    end

lights_on = false

false

In [64]:
@test begin  
    backend(autocad)
    
    alfa_init = 0
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
     
    no_doors = true
    lights_on = true
    
    delete_all_shapes()
    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    rooms_floor(u0(), 0, alfa_init, alfa_end, ri, re, corridor_w, floor_h, p_offset, n_pillars, m_pillars, 0.1)
    lights_rooms_floor(u0(), 0, alfa_init, alfa_end, ri, re, floor_h, p_offset, n_pillars, m_pillars)
end;

Expected result:
<img src="./plots/lights_plan.png" width="500">

# Complete floor plan

This function creates the complete floor plan for __floor zero__, which has larger rooms (auditoriums in this case) than the rest, and only one row of rooms as well. The function also joins in the lights defined above:

In [65]:
floor_zero(center, z, alfa_init, alfa_proj, alfa_end, inner_r, outer_r, corridor_w, p_offset, n_pillars, m_pillars, h, wall_thick)=
    let pillar_rs = division(inner_r+p_offset, outer_r-pillar_offset, m_pillars-1)
        r_in = pillar_rs[2]
        door_w = 1

        amp_tot = alfa_end-alfa_init
        amp_circle = alfa_proj-alfa_init
        amp_room = amp_tot/n_pillars
        proportion = amp_circle/amp_tot
        n_rooms = floor(n_pillars*proportion)
        amp_wall_area = alfa_init+n_rooms*amp_room
        n_walls = floor(Int, proportion*n_pillars/2)

        ang_dists = division(0, ang_dist(r_in, n_rooms*amp_room), n_walls)
        door_locs_1 = [x(ang+wall_thick/2) for ang in ang_dists[1:end-1]]
        door_locs_2 = [x(ang-door_w-wall_thick/2) for ang in ang_dists[2:end]]

        wall_trans_floor(center, z, alfa_init, amp_wall_area, inner_r, outer_r, corridor_w, h, p_offset, n_walls, m_pillars, broken=false)
        doors(curve_wall(center, z, alfa_init, amp_wall_area, r_in, h), [door_locs_1..., door_locs_2...])
        lights_on ? lights_rooms_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, h, p_offset, n_pillars, m_pillars) : nothing
    end

floor_zero (generic function with 1 method)

In [66]:
@test begin  
    backend(autocad)
    
    alfa_init = 0
    alfa_proj = pi+pi/4
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
     
    lights_on = true
    no_doors = false
    
    delete_all_shapes()
    isenberg_slab(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.1, 30)
    floor_zero(u0(), 0, alfa_init, alfa_proj, alfa_end, ri, re, corridor_w, p_offset, n_pillars, m_pillars, floor_h, 0.4)
end;

Expected result:
<img src="./plots/lights_plan_zero.png" width="500">

This function joins the complete floor plan defined above for the upper floors in `room_floor` and joins the lights:

In [67]:
floor_plans(center, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, floors, p_offset, n_pillars, m_pillars, wall_thick)=
    for z in division(h, h*floors, floors-1, false)
        rooms_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, corridor_w, h, p_offset, n_pillars, m_pillars, wall_thick)
        lights_on ? lights_rooms_floor(center, z, alfa_init, alfa_end, inner_r, outer_r, h, p_offset, n_pillars, m_pillars) : nothing
    end

floor_plans (generic function with 1 method)

In [68]:
@test begin  
    backend(meshcat)
    new_backend()
    
    alfa_init = 0
    alfa_proj = pi+pi/4
    alfa_end = 3pi/2
    ri = 5
    re = 15
    floor_h = 2.5
    n_pillars = 20
    m_pillars = 4
    p_offset = 1
    corridor_w = 2
    floors = 3
     
    lights_on = false
    no_doors = true
    
    delete_all_shapes()
    
    slabs(u0(), ri, re, alfa_init, alfa_end, alfa_end, 0.4, floor_h, floors, 30)
    straight_glass(u0(), ri, alfa_init, alfa_end, floor_h, floors, 30)
    straight_glass(u0(), re, alfa_init, alfa_end, floor_h, floors, 30)
    pillars_z(u0(), ri+p_offset, re-p_offset, alfa_init, alfa_end, n_pillars, m_pillars, floor_h, floors)
    
    floor_plans(u0(), alfa_init, alfa_end, ri, re, corridor_w, floor_h, floors, p_offset, n_pillars, m_pillars, 0.4)
    floor_zero(u0(), 0, alfa_init, alfa_proj, alfa_end, ri, re, corridor_w, p_offset, n_pillars, m_pillars, floor_h, 0.4)
end;

Expected result:
<img src="./plots/complete_floors.png" width="800">

# Site

This function creates a large plane (slab) to set the ground floor for the building. Change the `ground_fam` material in each specific backend to obtain different textures, e.g., grass, concrete, or simply white for clean renders of the building alone.

In [69]:
ground() =
    let x = 1000
        y = 1000
      slab(closed_polygonal_path([xy(-x,y), xy(x,y), xy(x,-y), xy(-x,-y)]), level(-0.05), ground_fam)
    end

ground (generic function with 1 method)

__Patio Trees__

This section contains the definition of random tree-like shapes to populate the surroundings of the building. Some unnecessary complexity can be seen in this code on account of Unity's unique handle on material attribution for non-BIM elements (such as the case of the cone-frustum which is not a BIM object). Khepri is evolving and this will eventually be evened out for all backends.

In [70]:
leaf(p, r, mat) =
    if current_backend() == unity
        with(current_material, get_material(mat)) do
            sphere(p, r)
        end
    else
        sphere(p, r)
    end

leaf (generic function with 1 method)

In [71]:
branch(pb, rb, pt, rt, mat)=
    if current_backend() == unity
        with(current_material, get_material(mat)) do
            cone_frustum(pb, rb, pt, rt)
        end
    else
        cone_frustum(pb, rb, pt, rt)
    end

branch (generic function with 1 method)

Three possible materials/colors were picked for the sphere-like leaves (green, purple, and yellow). These will only be visible in Unity. These colors should be defined in the global variable `leaves_color`, provided as a string.

In [72]:
tree(p_base, r_base, w_branch, fi, psi, min_f_w, max_f_w, max_fi, min_psi, max_psi) =
  let psi = psi > pi/2 ? psi - pi/2 : psi < -pi/2 ? psi + pi/2 : psi
      p_top = p_base + vsph(w_branch, fi, psi)
      r_top = r_base/2
      leaf_mat = leaves_color == "green" ? "Default/Materials/Grass" :
                leaves_color == "purple" ? "prefabs/bookshelves/dependencies/Patterns/GreenFabric" :
                "prefabs/bookshelves/dependencies/Patterns/DirtyCraftPaper" #yellow
      branch_mat = "materials/wood/ExteriorWood9"
      branch(p_base, r_base, p_top, r_top, branch_mat)
      if w_branch < 0.01 || r_top < 0.001
          winter ? nothing : leaf(p_top, 0.05, leaf_mat)
      else
          tree(p_top, r_top, w_branch*random_range(min_f_w, max_f_w), fi + random(max_fi), psi + random_range(min_psi, max_psi), min_f_w, max_f_w, max_fi, min_psi, max_psi)
          tree(p_top, r_top, w_branch*random_range(min_f_w, max_f_w), fi - random(max_fi), psi - random_range(min_psi, max_psi), min_f_w, max_f_w, max_fi, min_psi, max_psi)
      end
  end

tree (generic function with 1 method)

This function defines minimum and maximum values for the `tree` parameters, in order to spare the user the task of providing them for each tree. Also for a forest-like effect, randomness is a good option.

In [73]:
random_tree(p)=
  let r_base = random_range(0.05, 0.3)
      w_branch = random_range(2, 5)
      fi = 0
      psi = 0
      min_f_w = 0.6 # branch width max factor
      max_f_w = 0.9 # branch width min factor
      max_fi = 2pi
      min_psi = pi/16
      max_psi = pi/2
      tree(p, r_base, w_branch, fi, psi, min_f_w, max_f_w, max_fi, min_psi, max_psi)

end

random_tree (generic function with 1 method)

In [74]:
@test begin
    backend(unity)
    delete_all_shapes()
    
    set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))
    ground()
    winter=true
    tree(x(0), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)
    tree(x(5), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)
    tree(x(10), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)

#     render_dir("C:\\Users\\Renata\\Google Drive\\Khepri_Projects\\Isenberg_jl")
#     render_view("test")
end

| Unity render of the winter trees |  
|-------------------------------------------------------| 
| <img src="./figures/tree_winter.png" width="600"> |

In [75]:
@test begin
    backend(unity)
    delete_all_shapes()
    
    set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))
    ground()
    winter=false
    leaves_color = "yellow"
    tree(x(0), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)
    leaves_color = "green"
    tree(x(5), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)
    leaves_color = "purple"
    tree(x(10), .1, 2, 0, 0, .6, .9, 2pi, pi/16, pi/2)

#     render_dir("C:\\Users\\Renata\\Google Drive\\Khepri_Projects\\Isenberg_jl")
#     render_view("test")
end

| Unity render of the leaves' coloring |  
|-------------------------------------------------------| 
| <img src="./figures/trees_3_colors.png" width="600"> |

This function creates a small round forest of `random_tree`s. The circular forest has `p` as its center location, `r` for radius, `fi` for the angle covered by the forest (may not be a full circle), and `n` defines the number of trees in it. 

In [76]:
forest(p, r, fi, n)=
  for i in 0:n
      random_tree(p+vpol(random_range(2, r), random(fi)))
  end

forest (generic function with 1 method)

`choice` function chooses one random element from any given array.

In [77]:
function choice(a::Array)
  n = length(a)
  idx = rand(1:n)
  return a[idx]
end

choice (generic function with 1 method)

In [78]:
@test begin
    backend(meshcat)
    new_backend()
    winter = false
    leaves_color = choice(["green", "purple", "yellow"]) #random_element
    @manipulate for r = widget(1:1:10, label="Radius"),
                    fi = widget(0:pi/20:3pi/2, label="Max Angle"),
                    n = widget(1:1:10, label="Trees")
                    delete_all_shapes()
                    forest(u0(), r, fi, n)
    end
end

Expected result:

<img src="./plots/trees.png" width="800">

This function joins all site elements (not part of the construction itself), the ground and the patio trees:

In [79]:
function site()
  current_backend() == autocad || current_backend() == meshcat ? nothing : ground()
  current_backend() == meshcat ? nothing : forest(b_center, inner_radius-2, 2pi, n_trees)
end

site (generic function with 1 method)

# Generate Isenberg

The `isenberg` function contains no parameters. All isenberg variables should be defined as global variables. A list of the parameters required is provided in the ensuing section. use them as defaults and change the ones you like to create building variations. Some variation examples are also provided below.

In [80]:
isenberg()=
    let b_height = floor_height*n_floors
        outer_radius = inner_radius + building_width
        n_small_beams = ceil(Int, n_beams_total*2/3)
        n_large_beams = ceil(Int, n_beams_total/4)
        end_circle = begin_projection + pi/2

        slabs(b_center, inner_radius, outer_radius, begin_circle, begin_projection, end_circle, slab_thickness, floor_height, n_floors, n_crv_pts)
        outer_beams(b_center, b_height, outer_radius, begin_circle, begin_projection, n_small_beams)
        large_outer_beams(b_center, b_height, outer_radius, begin_projection, end_circle, beam_width, l_beam_lenght, n_large_beams)

        isenberg_wall(b_center+vpol(inner_radius, begin_circle), b_center+vpol(outer_radius, begin_circle), b_height, wall_thickness)
        isenberg_wall(b_center+vpol(inner_radius, end_circle), b_center+vpol(outer_radius, end_circle), b_height, wall_thickness)

        pillars_z(b_center, inner_radius+pillar_offset, outer_radius-pillar_offset, begin_circle, end_circle, n_pillars, m_pillars, floor_height, n_floors)
        floor_plans(b_center, begin_circle, end_circle, inner_radius, outer_radius, corridor_width, floor_height, n_floors, pillar_offset, n_pillars, m_pillars, wall_thickness)
        floor_zero(b_center, 0, begin_circle, begin_projection, end_circle, inner_radius, outer_radius, corridor_width, pillar_offset, n_pillars, m_pillars, floor_height, wall_thickness)

        straight_glass(b_center, inner_radius, begin_circle, end_circle, floor_height, n_floors, n_small_beams) # patio
        straight_glass(b_center+vz(floor_height), outer_radius, begin_projection, end_circle, floor_height, n_floors-1, n_large_beams*2) # inside entrance
        straight_glass(b_center, outer_radius, begin_circle, begin_projection, floor_height, n_floors, n_small_beams) # outside
        triangle_glass(b_center, b_height, outer_radius, begin_projection, end_circle, beam_width, l_beam_lenght, n_large_beams)
        large_beam_glass(b_center, b_height, outer_radius, begin_projection, end_circle, beam_width, l_beam_lenght, n_large_beams)
    end

isenberg (generic function with 1 method)

## Defaut parameters

In [81]:
# building shape
b_center = u0()
floor_height = 3
n_floors = 3
inner_radius = 10
building_width = 15

# Dimentions
slab_thickness = .4
wall_thickness = .4
#glass_thick = 0.05
beam_width = .4
s_beam_lenght = .4
l_beam_lenght = 3.5

# n points, n beams
n_beams_total = 60
n_crv_pts = 100

# Pilars and inside walls
n_pillars = n_beams_total/3
m_pillars = 4
pillar_offset = 1
corridor_width = 2

# Circle rotation
begin_circle = 0
begin_projection = pi

# Patio tress
n_trees = 15
winter = false
leaves_color = choice(["green", "purple", "yellow"])

# Booleans for Unity
no_doors = true
lights_on = false

false

In [82]:
@test begin
    backend(meshcat)
    new_backend()
    sleep(5)
    isenberg()
end

Expected result:

| | Complete Isenberg model in the Meshcat backend  |
|----------------------------------------------------|--|
| <img src="./plots/isen_meshcat1.png" width="450"> | <img src="./plots/isen_meshcat3.png" width="450"> |

| |  |
|----------------------------------------------------|--|
| <img src="./plots/isen_meshcat2.png" width="300"> | <img src="./plots/isen_meshcat4.png" width="600"> 

## Backend specifics

### Unity materials

Raw materials:

| Patined copper   | Rusted copper   | Concrete       | Plaster     | Steel           | Grass     |      
|------------------|-----------------|----------------|-------------|-----------------|-----------|
| <img src="./materials/copper_patina.png" width="130"> | <img src="./materials/copper_rusted.png" width="130"> | <img src="./materials/concrete.jpg" width="130"> | <img src="./materials/plaster.jpg" width="130"> | <img src="./materials/steel.jpg" width="130"> | <img src="./materials/grass.png" width="130"> |

| Copper plate | Copper bricks | Plaster filling | Steel | Aluminum |       
|--------------|---------------|-----------------|-------|----------|
| <img src="./materials/copper.png" width="150"> | <img src="./materials/copper_bricks.png" width="150"> | <img src="./materials/plaster2.png" width="150"> | <img src="./materials/steel.png" width="150"> | <img src="./materials/aluminum.png" width="150"> |

| Dark wood | Grass ground | Blue leaves | Purple leaves | Yellow leaves |     
|--------------|---------------|-----------------|-------|----------|
| <img src="./materials/wood9.png" width="150"> | <img src="./materials/grass2.png" width="150"> | <img src="./materials/color_blue.png" width="150"> | <img src="./materials/color_purple.png" width="150"> | <img src="./materials/color_yellow.png" width="150"> |

We use the chosen BIM families to attribute materials in the backend:

In [83]:
# -- default walls -- plaster
set_backend_family(default_wall_family(), unity, unity_material_family("Default/Materials/Plaster"))
# -- default slabs -- plaster
set_backend_family(default_slab_family(), unity, unity_material_family("Default/Materials/Plaster"))
# -- default columns -- copper bricks
set_backend_family(default_column_family(), unity, unity_material_family("materials/metal/CopperBricks"))
# -- default panels -- blue glass
set_backend_family(default_panel_family(), unity, unity_material_family("Default/Materials/GlassBlue"))
# --default curtain wall panels -- blue glass
set_backend_family(default_curtain_wall_family().panel, unity, unity_material_family("Default/Materials/GlassBlue"))

# -- exterior walls -- copper bricks
set_backend_family(copper_wall_fam, unity, unity_material_family("materials/metal/CopperBricks"))
# -- metal window frames -- steel 
set_backend_family(frame_fam, unity, unity_material_family("Default/Materials/Steel"))
# -- inner columns/pillars -- plaster
set_backend_family(pillar_fam, unity, unity_material_family("Default/Materials/Plaster"))
# -- doors -- glass
set_backend_family(door_fam, unity, unity_material_family("Default/Materials/Glass"))

# -- ground -- grass
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/Grass"))
# -- ground -- white plane with shadows
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/White"))
# -- ground -- white plane, no shadows
set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))

IdDict{Backend,Any} with 0 entries

## Variations

In [84]:
@test begin
    backend(unity)
    
    delete_all_shapes()
    ground()
    isenberg()
    
    b_center = x(90)
    floor_height = 4
    inner_radius = 15
    building_width = 16
    l_beam_lenght = 4.2
    n_beams_total = 60
    begin_circle = pi/4
    begin_projection = pi-pi/4
    isenberg()

    b_center = x(160)
    floor_height = 2.8
    n_floors = 5
    inner_radius = 10
    building_width = 20
    l_beam_lenght = 3.9
    n_beams_total = 90
    begin_circle = pi/4
    begin_projection = pi/2
    isenberg()

    b_center = x(-80)
    n_floors = 2
    l_beam_lenght = 2.3
    inner_radius = 12
    building_width = 12
    n_beams_total = 60
    begin_circle = pi/10
    begin_projection = 3pi/2
    isenberg()
end

| Isenberg variations section vew                      |          
|------------------------------------------------------|
| <img src="./figures/render_section.png" width="900"> | 

| Isenberg variations plan view                     |
|---------------------------------------------------|
| <img src="./figures/render_plan.png" width="900"> |

In [85]:
@test begin
    backend(unity)
    
    delete_all_shapes()
    set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))
    ground()
    
#     inner_radius = 10
#     n_floors = 3
    inner_radius = 5
    n_floors = 5
    
    isenberg()
    
end

| Isenberg inner radius and number of floors variation     |
|---------------------------------------------------|
| <img src="./figures/raio_5_10_Npisos_3_5.png" width="900"> |

In [86]:
@test begin
    backend(unity)
    
    delete_all_shapes()
    set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit"))
    ground()
    
#     n_beams_total = 60
#     n_floors = 3
#     l_beam_lenght = 3.5
    n_floors = 5
    n_beams_total = 40
    l_beam_lenght = 6
    
    isenberg()
    
end

| Isenberg number of large beams and floors variation |
|---------------------------------------------------|
| <img src="./figures/vars_2_qual.png" width="900"> |

In [87]:
@test begin
    backend(unity)
    
    winter = false # true
    leaves_color = "green" # "purple" "yellow"
    
    delete_all_shapes()
    site()
    isenberg()
    
end

__Unity renders of patio trees__

| without leaves | with green leaves |
|----------------|-------------------|
| <img src="./figures/t4.png" width="450"> | <img src="./figures/t1.png" width="450"> |

| with purple leaves | with yellow leaves |
|--------------------|--------------------|
| <img src="./figures/t2.png" width="450"> | <img src="./figures/t3.png" width="450"> |

_The End_